# DATA PRE-PROCESSING NOTEBOOK

### **I. Import thư viện**

In [53]:
import os
import glob
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

### **II. Nối các file csv thu thập được thành một file duy nhất**

Thiết lập đường dẫn folder chứa các file csv input và tên file csv output sau khi nối

In [54]:
INPUT_FOLDER = "data"  
OUTPUT_FOLDER = "output"
RAW_FILENAME = "football_players_raw_dataset.csv"
FINAL_FILENAME = "football_players_dataset.csv"

RAW_OUTPUT_FILE = os.path.join(OUTPUT_FOLDER, "football_players_raw_dataset.csv")
FINAL_OUTPUT_FILE = os.path.join(OUTPUT_FOLDER, "football_players_dataset.csv")

In [55]:
def merge_csv(folder_path, output_folder, output_filename, drop_duplicate=True):
    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, output_filename)

    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    if not all_files:
        print("Not found any CSV files in the folder.")
        return
    
    dfs = []

    for file in all_files:
        try:
            df = pd.read_csv(file)
            print(f"Loaded: {file} ({len(df)} rows)")
            dfs.append(df)
        except Exception as e:
            print(f"Read error for file {file}: {e}")

    merged_df = pd.concat(dfs, ignore_index=True)

    if drop_duplicate:
        before = len(merged_df)
        merged_df = merged_df.drop_duplicates(subset="player_id", keep="last")
        after = len(merged_df)
        print(f"Removed {before - after} duplicates")

    merged_df.to_csv(output_path, index=False)
    print(f"Merged file saved at: {output_path}")

In [56]:
merge_csv(INPUT_FOLDER, OUTPUT_FOLDER, RAW_FILENAME)

Loaded: data\A-League_Men.csv (378 rows)
Loaded: data\Allsvenskan.csv (523 rows)
Loaded: data\Argentine_Liga.csv (1274 rows)
Loaded: data\Austrian_Bundesliga.csv (390 rows)
Loaded: data\Belgian_Pro_League.csv (578 rows)
Loaded: data\Bolivian_Primera_División.csv (115 rows)
Loaded: data\Bundesliga.csv (578 rows)
Loaded: data\Canadian_Premier_League.csv (13 rows)
Loaded: data\Challenger_Pro_League.csv (515 rows)
Loaded: data\Chilean_Primera_División.csv (198 rows)
Loaded: data\Chinese_Super_League.csv (525 rows)
Loaded: data\Croatian_Football_League.csv (108 rows)
Loaded: data\Czech_First_League.csv (619 rows)
Loaded: data\Danish_Superliga.csv (159 rows)
Loaded: data\Eerste_Divisie.csv (25 rows)
Loaded: data\EFL_League_One.csv (270 rows)
Loaded: data\EFL_League_Two.csv (105 rows)
Loaded: data\Ekstraklasa.csv (2 rows)
Loaded: data\Eliteserien.csv (523 rows)
Loaded: data\Eredivisie.csv (638 rows)
Loaded: data\football_players_full_dataset.csv (20682 rows)
Loaded: data\Hrvatska_NL.csv (415 

### **III. Tiền xử lí dữ liệu**

#### **1. Đọc và khám phá dữ liệu**

In [57]:
df = pd.read_csv(RAW_OUTPUT_FILE)
print(f"\nTập dữ liệu: {df.shape[0]} players, {df.shape[1]} features")


print("\nThông tin cơ bản:")
print(f"   - Giá trị thiếu (Missing values): {df.isnull().sum().sum():,} cells")
print(f"   - Bộ nhớ sử dụng (Memory usage): {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")



Tập dữ liệu: 20682 players, 70 features

Thông tin cơ bản:
   - Giá trị thiếu (Missing values): 764,870 cells
   - Bộ nhớ sử dụng (Memory usage): 19.17 MB


#### **2. Phân loại vị trí cầu thủ**

In [58]:
print("PHÂN LOẠI VỊ TRÍ CẦU THỦ")

def classify_position(pos):
    """Phân loại vị trí cầu thủ thành 4 nhóm chính"""
    if pd.isna(pos):
        return 'UNKNOWN'
    pos = str(pos).upper()
    if 'GK' in pos:
        return 'GK'
    elif any(x in pos for x in ['DF', 'CB', 'LB', 'RB', 'WB']):
        return 'DF'
    elif any(x in pos for x in ['MF', 'CM', 'DM', 'AM']):
        return 'MF'
    elif any(x in pos for x in ['FW', 'ST', 'CF', 'LW', 'RW']):
        return 'FW'
    else:
        return 'UNKNOWN'

df['position_category'] = df['position'].apply(classify_position)

print("\n- Phân bố vị trí:")
for pos, count in df['position_category'].value_counts().items():
    pct = count / len(df) * 100
    print(f"   {pos:8s}: {count:4d} cầu thủ ({pct:5.1f}%)")

# Tạo indicator columns
position_categories = ['GK', 'DF', 'MF', 'FW']
for pos in position_categories:
    df[f'is_{pos}'] = (df['position_category'] == pos).astype(int)

PHÂN LOẠI VỊ TRÍ CẦU THỦ

- Phân bố vị trí:
   MF      : 8136 cầu thủ ( 39.3%)
   DF      : 7192 cầu thủ ( 34.8%)
   FW      : 3193 cầu thủ ( 15.4%)
   GK      : 2138 cầu thủ ( 10.3%)
   UNKNOWN :   23 cầu thủ (  0.1%)


#### **3. Phân tích Missing Values**

In [59]:
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Pct': (df.isnull().sum().values / len(df) * 100)
})
missing_analysis['Data_Type'] = missing_analysis['Column'].apply(lambda x: str(df[x].dtype))
missing_analysis = missing_analysis.sort_values('Missing_Pct', ascending=False)

print("\nCÁC CỘT CÓ MISSING > 5%:")
print("-" * 80)
high_missing = missing_analysis[missing_analysis['Missing_Pct'] > 5]

for idx, row in high_missing.head(20).iterrows():
    status = "🔴" if row['Missing_Pct'] > 80 else "🟡" if row['Missing_Pct'] > 50 else "🟢"
    print(f"{status} {row['Column']:45s}: {row['Missing_Pct']:6.2f}%")


CÁC CỘT CÓ MISSING > 5%:
--------------------------------------------------------------------------------
🔴 penalty_save_pct                             :  96.85%
🔴 avg_pass_length                              :  95.81%
🔴 wins_per90                                   :  95.81%
🔴 def_actions_outside_pen_per90                :  95.81%
🔴 shots_on_target_against_per90                :  95.81%
🔴 saves_per90                                  :  95.81%
🔴 save_percentage                              :  95.81%
🔴 avg_distance_def_actions                     :  95.81%
🔴 crosses_stopped_pct                          :  95.81%
🔴 clean_sheet_pct                              :  95.81%
🔴 goals_against_per90                          :  95.81%
🔴 draws_per90                                  :  95.81%
🔴 losses_per90                                 :  95.81%
🔴 psxg_per_shot                                :  95.81%
🔴 psxg_ga_per90                                :  95.81%
🔴 launch_pct                          

#### **4. Phân tích Correlation - Phát hiện trùng lặp**

In [60]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
exclude_from_corr = ['market_value'] + [f'is_{pos}' for pos in position_categories]
numeric_cols_for_corr = [c for c in numeric_cols if c not in exclude_from_corr]

# Tính correlation matrix
corr_matrix = df[numeric_cols_for_corr].corr()

# Tìm các cặp có correlation cao (> 0.9)
print("\nCÁC CẶP CỘT CÓ CORRELATION CAO (> 0.90):")
print("-" * 80)

high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.90:
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]
            high_corr_pairs.append({
                'Column_1': col1,
                'Column_2': col2,
                'Correlation': corr_val
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', 
                                                              ascending=False, 
                                                              key=abs)
    for idx, row in high_corr_df.iterrows():
        emoji = "🔴" if abs(row['Correlation']) > 0.95 else "🟡"
        print(f"{emoji} {row['Column_1']:35s} <-> {row['Column_2']:35s}: {row['Correlation']:7.4f}")
else:
    print("- Không có cặp nào có correlation > 0.90")


CÁC CẶP CỘT CÓ CORRELATION CAO (> 0.90):
--------------------------------------------------------------------------------
🔴 goals_per_90                        <-> def_actions_outside_pen_per90      : -0.9995
🔴 goals_per_90                        <-> saves_per90                        :  0.9938
🔴 goals_per_90                        <-> avg_distance_def_actions           : -0.9916
🔴 goals_per_90                        <-> clean_sheet_pct                    :  0.9881
🔴 launch_pct                          <-> avg_pass_length                    :  0.9771
🔴 goals_per_90                        <-> losses_per90                       :  0.9732
🔴 npxg_per90                          <-> xg_per90                           :  0.9656
🔴 passes_completed_per90              <-> touches_per90                      :  0.9612
🟡 goals_per_90                        <-> psxg_ga_per90                      :  0.9497
🟡 carries_per90                       <-> passes_received_per90              :  0.9496
🟡 appea

#### **5. Phân tích Correlation với Target**

In [61]:
target_corr = {}
for col in numeric_cols_for_corr:
    valid_data = df[[col, 'market_value']].dropna()
    if len(valid_data) > 30:
        corr = valid_data[col].corr(valid_data['market_value'])
        target_corr[col] = corr

# Sắp xếp theo absolute correlation
target_corr_sorted = sorted(target_corr.items(), 
                           key=lambda x: abs(x[1]), 
                           reverse=True)

print("\n- TOP 15 FEATURES TƯƠNG QUAN CAO NHẤT VỚI MARKET_VALUE:")
print("-" * 80)
for i, (col, corr) in enumerate(target_corr_sorted[:15], 1):
    emoji = "📈" if corr > 0 else "📉"
    strength = "Strong: " if abs(corr) > 0.5 else "Medium" if abs(corr) > 0.3 else "  "
    print(f"{i:2d}. {strength} {emoji} {col:45s}: {corr:7.4f}")


- TOP 15 FEATURES TƯƠNG QUAN CAO NHẤT VỚI MARKET_VALUE:
--------------------------------------------------------------------------------
 1.    📈 goals                                        :  0.2567
 2.    📈 assists                                      :  0.2418
 3.    📈 wins_per90                                   :  0.2282
 4.    📈 minutes_played                               :  0.2027
 5.    📈 appearances                                  :  0.2005
 6.    📈 passes_received_per90                        :  0.1933
 7.    📈 touches_att_pen_per90                        :  0.1878
 8.    📈 npxg_xag_per90                               :  0.1865
 9.    📈 gca_per90                                    :  0.1777
10.    📈 touches_att_third_per90                      :  0.1768
11.    📉 losses_per90                                 : -0.1755
12.    📈 carries_per90                                :  0.1691
13.    📉 goals_against_per90                          : -0.1662
14.    📈 npg_per90            

#### **6. Phân tích VIF (Multicollinearity)**

In [62]:
cols_for_vif = [col for col in numeric_cols_for_corr 
                if df[col].isnull().sum() / len(df) < 0.5][:30]

df_vif = df[cols_for_vif].dropna()

if len(df_vif) > 10 and len(cols_for_vif) > 1:
    print(f"\n- Tính VIF trên {len(df_vif)} samples với {len(cols_for_vif)} features")
    print("(VIF > 10: multicollinearity cao, VIF > 100: cực kỳ cao)")
    print("-" * 80)
    
    try:
        vif_data = pd.DataFrame()
        vif_data["Feature"] = cols_for_vif
        vif_data["VIF"] = [variance_inflation_factor(df_vif.values, i) 
                          for i in range(len(cols_for_vif))]
        vif_data = vif_data.sort_values('VIF', ascending=False)
        
        # Hiển thị top 15
        print("\nTOP 15 VIF CAO NHẤT:")
        for idx, row in vif_data.head(15).iterrows():
            if row['VIF'] > 100:
                status = "🔴🔴"
            elif row['VIF'] > 50:
                status = "🔴  "
            elif row['VIF'] > 10:
                status = "🟡  "
            else:
                status = "✅  "
            print(f"{status} {row['Feature']:45s}: {row['VIF']:10.2f}")
        
        # Thống kê
        print(f"\nTHỐNG KÊ VIF:")
        print(f"   - VIF > 100 (Cực cao): {len(vif_data[vif_data['VIF'] > 100])} cột")
        print(f"   - VIF 10-100 (Cao): {len(vif_data[(vif_data['VIF'] > 10) & (vif_data['VIF'] <= 100)])} cột")
        print(f"   - VIF < 10 (OK): {len(vif_data[vif_data['VIF'] <= 10])} cột")
        
    except Exception as e:
        print(f"- Không thể tính VIF: {str(e)}")
        vif_data = None
else:
    print("-  Không đủ dữ liệu để tính VIF")
    vif_data = None


- Tính VIF trên 11035 samples với 9 features
(VIF > 10: multicollinearity cao, VIF > 100: cực kỳ cao)
--------------------------------------------------------------------------------

TOP 15 VIF CAO NHẤT:
🔴   height                                       :      62.69
🔴   minutes_played                               :      54.68
🟡   appearances                                  :      48.69
🟡   age                                          :      38.22
🟡   minutes_per_game                             :      36.53
✅   goals                                        :       4.05
✅   assists                                      :       3.87
✅   goals_per_90                                 :       3.67
✅   assists_per_90                               :       2.92

THỐNG KÊ VIF:
   - VIF > 100 (Cực cao): 0 cột
   - VIF 10-100 (Cao): 5 cột
   - VIF < 10 (OK): 4 cột


#### **7. Phân tích các cột dẫn xuất**

In [63]:
print("\nKiểm tra các cột có thể tính từ cột khác:")
print("-" * 80)

derived_candidates = []

# Check minutes_per_game
if all(col in df.columns for col in ['minutes_per_game', 'minutes_played', 'appearances']):
    df['calculated_mpg'] = df['minutes_played'] / df['appearances'].replace(0, 1)
    valid = df[['minutes_per_game', 'calculated_mpg']].dropna()
    if len(valid) > 10:
        correlation = valid['minutes_per_game'].corr(valid['calculated_mpg'])
        if abs(correlation) > 0.99:
            derived_candidates.append({
                'Column': 'minutes_per_game',
                'Derived_From': 'minutes_played / appearances',
                'Correlation': correlation
            })
            print(f"✓ minutes_per_game = minutes_played / appearances (corr: {correlation:.4f})")

# Check các metrics tổng hợp
potential_sums = [
    ('npxg_xag_per90', ['npxg_per90', 'xag_per90']),
]

for target_col, source_cols in potential_sums:
    if target_col in df.columns and all(col in df.columns for col in source_cols):
        df['calculated_sum'] = df[source_cols].sum(axis=1)
        valid_data = df[[target_col, 'calculated_sum']].dropna()
        if len(valid_data) > 10:
            correlation = valid_data[target_col].corr(valid_data['calculated_sum'])
            if abs(correlation) > 0.95:
                derived_candidates.append({
                    'Column': target_col,
                    'Derived_From': ' + '.join(source_cols),
                    'Correlation': correlation
                })
                print(f"✓ {target_col} = {' + '.join(source_cols)} (corr: {correlation:.4f})")

if not derived_candidates:
    print("- Không tìm thấy cột derived rõ ràng")


Kiểm tra các cột có thể tính từ cột khác:
--------------------------------------------------------------------------------
✓ minutes_per_game = minutes_played / appearances (corr: 1.0000)
✓ npxg_xag_per90 = npxg_per90 + xag_per90 (corr: 0.9987)


#### **8. Tạo Recommendation**

In [64]:
recommendations = {
    'high_missing': [],
    'low_correlation': [],
    'high_vif': [],
    'derived': [],
    'duplicate': [],
    'metadata': []
}

# 1. Missing quá cao (> 90%)
for idx, row in missing_analysis.iterrows():
    if row['Missing_Pct'] > 90:
        recommendations['high_missing'].append({
            'column': row['Column'],
            'reason': f"Missing {row['Missing_Pct']:.1f}%",
            'priority': 'HIGH'
        })

# 2. Correlation thấp với target (< 0.03)
for col, corr in target_corr.items():
    if abs(corr) < 0.03:
        recommendations['low_correlation'].append({
            'column': col,
            'reason': f"Low correlation: {corr:.4f}",
            'priority': 'LOW'
        })

# 3. VIF cao
if vif_data is not None:
    for idx, row in vif_data.iterrows():
        if row['VIF'] > 70:
            recommendations['high_vif'].append({
                'column': row['Feature'],
                'reason': f"VIF = {row['VIF']:.1f}",
                'priority': 'HIGH'
            })

# 4. Derived columns
for item in derived_candidates:
    recommendations['derived'].append({
        'column': item['Column'],
        'reason': f"Derived from {item['Derived_From']}",
        'priority': 'MEDIUM'
    })

# 5. Duplicate (corr > 0.95)
for item in high_corr_pairs:
    if abs(item['Correlation']) > 0.95:
        corr1 = abs(target_corr.get(item['Column_1'], 0))
        corr2 = abs(target_corr.get(item['Column_2'], 0))
        
        to_drop = item['Column_1'] if corr1 < corr2 else item['Column_2']
        to_keep = item['Column_2'] if corr1 < corr2 else item['Column_1']
        
        recommendations['duplicate'].append({
            'column': to_drop,
            'reason': f"Duplicate với {to_keep} (corr: {item['Correlation']:.4f})",
            'priority': 'HIGH'
        })

# 6. Metadata
metadata_cols = ['player_id', 'player_name', 'foot']
for col in metadata_cols:
    if col in df.columns:
        recommendations['metadata'].append({
            'column': col,
            'reason': 'Metadata không cần cho modeling',
            'priority': 'MEDIUM'
        })

# In recommendations
print("\n- DANH SÁCH ĐỀ XUẤT LOẠI BỎ:")
print("="*80)

total_to_drop = 0
columns_to_drop = []

for category, items in recommendations.items():
    if items:
        print(f"\n🔸 {category.upper().replace('_', ' ')} ({len(items)} cột):")
        print("-" * 80)
        for item in items:
            priority_emoji = "🔴" if item['priority'] == 'HIGH' else "🟡" if item['priority'] == 'MEDIUM' else "🟢"
            print(f"{priority_emoji} {item['column']:45s}: {item['reason']}")
            columns_to_drop.append(item['column'])
            total_to_drop += 1

print(f"\n- TỔNG KẾT PHÂN TÍCH:")
print(f"   - Tổng số cột hiện tại: {len(df.columns)}")
print(f"   - Đề xuất loại bỏ: {total_to_drop} cột")
print(f"   - Còn lại dự kiến: {len(df.columns) - total_to_drop} cột")

# Lưu recommendations
all_recommendations = []
for category, items in recommendations.items():
    for item in items:
        all_recommendations.append({
            'Category': category,
            'Column': item['column'],
            'Reason': item['reason'],
            'Priority': item['priority']
        })


- DANH SÁCH ĐỀ XUẤT LOẠI BỎ:

🔸 HIGH MISSING (17 cột):
--------------------------------------------------------------------------------
🔴 penalty_save_pct                             : Missing 96.8%
🔴 avg_pass_length                              : Missing 95.8%
🔴 wins_per90                                   : Missing 95.8%
🔴 def_actions_outside_pen_per90                : Missing 95.8%
🔴 shots_on_target_against_per90                : Missing 95.8%
🔴 saves_per90                                  : Missing 95.8%
🔴 save_percentage                              : Missing 95.8%
🔴 avg_distance_def_actions                     : Missing 95.8%
🔴 crosses_stopped_pct                          : Missing 95.8%
🔴 clean_sheet_pct                              : Missing 95.8%
🔴 goals_against_per90                          : Missing 95.8%
🔴 draws_per90                                  : Missing 95.8%
🔴 losses_per90                                 : Missing 95.8%
🔴 psxg_per_shot                             

#### **9. Loại bỏ các cột**

In [65]:
columns_to_drop = list(set(columns_to_drop))
columns_to_drop_exist = [col for col in columns_to_drop if col in df.columns]

df_cleaned = df.drop(columns=columns_to_drop_exist)

print(f"- Đã loại bỏ: {len(columns_to_drop_exist)} cột")
print(f"- Dataset sau khi làm sạch: {df_cleaned.shape}")

- Đã loại bỏ: 31 cột
- Dataset sau khi làm sạch: (20682, 46)


#### **10. Phân tích Missing Values theo vị trí**

In [66]:
POSITION_SPECIFIC_COLS = {
    'GK': [
        'goals_against_per90', 'shots_on_target_against_per90', 'saves_per90',
        'save_percentage', 'clean_sheet_pct', 'psxg_per_shot', 'psxg_ga_per90',
        'penalty_save_pct', 'launch_pct', 'avg_pass_length',
        'def_actions_outside_pen_per90', 'avg_distance_def_actions', 
        'crosses_stopped_pct'
    ],
    'DF': [
        'tackles_per90', 'interceptions_per90', 'blocks_per90',
        'ball_recoveries_per90', 'aerials_won_per90', 'aerial_win_pct'
    ],
    'MF': [
        'key_passes_per90', 'sca_per90', 'gca_per90',
        'xag_per90', 'assists_per_90'
    ],
    'FW': [
        'goals_per_90', 'assists_per_90', 'npg_per90', 
        'npxg_per90', 'xag_per90', 'sca_per90', 'gca_per90'
    ]
}

print("\n- Tỷ lệ missing của features theo vị trí:\n")

for pos in position_categories:
    df_pos = df_cleaned[df_cleaned['position_category'] == pos]
    
    if len(df_pos) == 0:
        continue
    
    print(f"\n{pos} ({len(df_pos)} cầu thủ):")
    print("-" * 70)
    
    if pos in POSITION_SPECIFIC_COLS:
        for col in POSITION_SPECIFIC_COLS[pos]:
            if col in df_cleaned.columns:
                missing_pct = df_pos[col].isna().sum() / len(df_pos) * 100
                status = "✅" if missing_pct < 30 else "⚠️" if missing_pct < 60 else "❌"
                print(f"   {status} {col:45s}: {missing_pct:5.1f}%")


- Tỷ lệ missing của features theo vị trí:


GK (2138 cầu thủ):
----------------------------------------------------------------------

DF (7192 cầu thủ):
----------------------------------------------------------------------
   ⚠️ interceptions_per90                          :  42.7%
   ⚠️ blocks_per90                                 :  42.7%
   ⚠️ ball_recoveries_per90                        :  42.7%
   ⚠️ aerials_won_per90                            :  42.7%

MF (8136 cầu thủ):
----------------------------------------------------------------------
   ⚠️ key_passes_per90                             :  44.4%
   ⚠️ sca_per90                                    :  44.4%
   ⚠️ gca_per90                                    :  44.4%
   ⚠️ xag_per90                                    :  44.4%

FW (3193 cầu thủ):
----------------------------------------------------------------------
   ❌ npg_per90                                    :  61.4%
   ❌ xag_per90                                    :  

#### **11. Xử lí Market Value Missing**

In [67]:
market_value_missing = df_cleaned['market_value'].isna().sum()
print(f"\n- Market value missing: {market_value_missing} ({market_value_missing/len(df_cleaned)*100:.1f}%)")

if market_value_missing > 0:
    df_cleaned = df_cleaned.dropna(subset=['market_value']).copy()
    print(f"Đã loại bỏ {market_value_missing} dòng")
    print(f"Dataset còn lại: {len(df_cleaned)} cầu thủ")


- Market value missing: 4229 (20.4%)
Đã loại bỏ 4229 dòng
Dataset còn lại: 16453 cầu thủ


#### **12. Impute missing values theo vị trí**

In [68]:
categorical_cols = ['nationality', 'position', 'position_category', 
                   'current_club', 'league']
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
exclude_from_impute = ['market_value'] + [f'is_{p}' for p in position_categories]
cols_to_impute = [c for c in numeric_cols if c not in exclude_from_impute]

print(f"\n- Impute {len(cols_to_impute)} numeric columns theo median của từng vị trí\n")

for pos in position_categories:
    mask = df_cleaned['position_category'] == pos
    df_pos = df_cleaned[mask]
    
    if len(df_pos) == 0:
        continue
    
    print(f"{pos}:")
    
    # Impute bằng median
    imputer = SimpleImputer(strategy='median')
    
    cols_with_data = [col for col in cols_to_impute 
                      if df_pos[col].notna().sum() > 0]
    
    before_missing = df_cleaned.loc[mask, cols_with_data].isnull().sum().sum()
    
    if cols_with_data:
        imputed_values = imputer.fit_transform(df_pos[cols_with_data])
        df_cleaned.loc[mask, cols_with_data] = imputed_values
    
    # Fill 0 cho cột toàn NULL
    cols_all_null = [col for col in cols_to_impute 
                     if df_pos[col].notna().sum() == 0]
    
    if cols_all_null:
        df_cleaned.loc[mask, cols_all_null] = 0
    
    after_missing = df_cleaned.loc[mask, cols_with_data].isnull().sum().sum()
    
    print(f"   - Imputed {len(cols_with_data)} columns")
    print(f"   - Missing: {before_missing:,} → {after_missing:,}")


- Impute 36 numeric columns theo median của từng vị trí

GK:
   - Imputed 7 columns
   - Missing: 30 → 0
DF:
   - Imputed 36 columns
   - Missing: 57,170 → 0
MF:
   - Imputed 36 columns
   - Missing: 68,430 → 0
FW:
   - Imputed 36 columns
   - Missing: 34,283 → 0


#### **13. Lưu tập dữ liệu**

In [69]:
# Sắp xếp cột
column_order = (
    ['age', 'height', 'nationality', 'position', 'position_category', 
     'current_club', 'league', 'appearances', 'minutes_played'] +
    [f'is_{pos}' for pos in position_categories] +
    ['age_group', 'experience_level', 'minutes_per_appearance']
)

numeric_features = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [c for c in numeric_features 
                   if c not in column_order and c != 'market_value']
column_order.extend(numeric_features)
column_order.append('market_value')

column_order = [col for col in column_order if col in df_cleaned.columns]
df_final = df_cleaned[column_order]

output_folder = os.path.dirname(FINAL_OUTPUT_FILE)
os.makedirs(output_folder, exist_ok=True)

# Lưu file
df_final.to_csv(FINAL_OUTPUT_FILE, index=False)

# In thông tin
print(f"\nSaved to: {FINAL_OUTPUT_FILE}")
print(f"   - Rows: {len(df_final):,}")
print(f"   - Columns: {len(df_final.columns)}")


Saved to: output\football_players_dataset.csv
   - Rows: 16,453
   - Columns: 46
